In [1]:
import numpy as np
import yfinance as yf
from math import sqrt
from pandas_datareader import data as pdr

In [2]:
eq = []
syms = ['SBIN.NS', 'AXISBANK.BO', 'IOC.BO', 'NMDC.BO', 'KRITIIND.BO']
for i in range(5):
    eq.append(pdr.get_data_yahoo(syms[i], start = '2016-07-02', end = '2019-07-02'))

#### Saving data in csvs

In [3]:
for i in range(5):
    eq[i].to_csv(f"{syms[i]}.csv", index = False)

In [4]:
eq[0].head()

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2016-07-04,224.350006,222.000000,222.350006,223.000000,16534518.0,221.001373
2016-07-05,225.149994,221.550003,223.250000,223.500000,15357018.0,221.496887
2016-07-07,224.600006,219.350006,223.899994,220.050003,12917872.0,218.077820
2016-07-08,220.850006,216.500000,220.149994,218.300003,16017389.0,216.343506
2016-07-11,225.449997,222.000000,223.600006,224.699997,15831791.0,222.686127


In [5]:
for e in eq:
    e['Daily Return'] = (e['Close'] - e['Open'])*100 / e['Open']

In [6]:
for e in eq:
    avg = np.mean(e['Daily Return'])
    e['Ret - avg'] = e['Daily Return'] - avg

In [7]:
m = eq[0].shape[0]
n = len(eq)
excess_return = np.zeros([m, n])
for i in range(n):
    excess_return[:, i] = eq[i]['Ret - avg']

In [8]:
X = excess_return
mat = np.dot(np.transpose(X), X)

In [9]:
vcm = mat / m

In [10]:
std = []
for e in eq:
    mean = np.mean(e['Daily Return'])
    m = e.shape[0]
    st = sqrt(np.sum((e['Daily Return'] - mean)**2) / m)
    std.append(st)
print(std)

[1.809366106279542, 1.718755260244544, 1.8463833426041016, 1.8301530732265505, 3.708787838215061]


In [11]:
stds = np.array(std).reshape(-1, 1)

In [12]:
product_std = np.dot(stds, np.transpose(stds))
product_std.shape

(5, 5)

### Correlation
**corr = variance_covariance_matrix / product_of_sds**

In [13]:
corr = vcm / product_std
print(corr)

[[1.         0.40853379 0.20050085 0.29950005 0.1232868 ]
 [0.40853379 1.         0.18705031 0.19956484 0.05505373]
 [0.20050085 0.18705031 1.         0.24496592 0.06460634]
 [0.29950005 0.19956484 0.24496592 1.         0.14394112]
 [0.1232868  0.05505373 0.06460634 0.14394112 1.        ]]


In [14]:
weights = np.array([0.2, 0.2, 0.2, 0.2, 0.2]).reshape(-1, 1)
weights.shape

(5, 1)

In [15]:
weighted_sd = stds * weights
print(weighted_sd, stds)

[[0.36187322]
 [0.34375105]
 [0.36927667]
 [0.36603061]
 [0.74175757]] [[1.80936611]
 [1.71875526]
 [1.84638334]
 [1.83015307]
 [3.70878784]]


### Portfolio Variance
**Portfolio Variance = Sqrt (Transpose (Wt.SD) * Correlation Matrix * Wt. SD)**

In [16]:
portfolio_var = np.dot(np.dot(np.transpose(weighted_sd), corr), weighted_sd)
portfolio_var[0][0]

1.6759776938960191

In [17]:
final_pv = sqrt(portfolio_var[0][0])
final_pv

1.2945955715573954

### Minimizing Portfolio Variance

#### 1. Monte-Carlo Simulations: Using randomly generated weights

In [18]:
port_vars = []
rand_wts = []
for i in range(0, 10000):
    random_weights = np.random.dirichlet(np.ones(5),size=1).T
    rand_wts.append(random_weights)
    random_weighted_sd = stds * random_weights
    portfolio_var = sqrt(np.sum(np.dot(np.dot(np.transpose(random_weighted_sd), corr), random_weighted_sd)))
    port_vars.append(portfolio_var)
min_index = port_vars.index(min(port_vars))
best_wts = rand_wts[min_index]
min_var = port_vars[min_index]
print(f"Best Weights: {best_wts}, Minimum Portfolio Variance: {min_var}, Sum of weights: {np.sum(best_wts)}")

Best Weights: [[0.14759249]
 [0.32536672]
 [0.25191417]
 [0.21104525]
 [0.06408138]], Minimum Portfolio Variance: 1.1756749637371504, Sum of weights: 0.9999999999999998


#### 2. By minimizing function of portfolio variance as a parameter of weights

In [19]:
from scipy.optimize import minimize

In [20]:
def portfolio_variance(weights, stds, corr):
    weighted_sd = weights * stds
    k = np.dot(np.transpose(weighted_sd), corr)
    ans = np.dot(k, weighted_sd)[0][0]
    return sqrt(ans)

In [21]:
cons = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
bnds = ((0.001, 1), (0.001, 1), (0.001, 1), (0.001, 1), (0.001, 1))

In [22]:
minimize(portfolio_variance, [0.3, 0.3, 0.2, 0.1, 0.1], args = (stds, corr),
         method = "SLSQP", constraints = cons, bounds = bnds)

     fun: 0.006472977858323092
     jac: array([6.47297786, 0.        , 0.        , 0.        , 0.        ])
 message: 'Optimization terminated successfully.'
    nfev: 14
     nit: 2
    njev: 2
  status: 0
 success: True
       x: array([0.001  , 0.37475, 0.27475, 0.17475, 0.17475])

In [23]:
min_op = minimize(portfolio_variance, [0.3, 0.3, 0.2, 0.1, 0.1], args = (stds, corr),
         method = "SLSQP", constraints = cons, bounds = bnds)

In [24]:
best_weights = min_op.x.reshape(-1, 1)

In [25]:
min_variance = min_op.fun

In [26]:
print(f"Best Weights to achieve minimum portfolio variance: {best_weights}")
print(f"Minimum value of porfolio variance: {float(min_variance)}")

Best Weights to achieve minimum portfolio variance: [[0.001  ]
 [0.37475]
 [0.27475]
 [0.17475]
 [0.17475]]
Minimum value of porfolio variance: 0.006472977858323092


In [27]:
print(np.sum(best_weights))

1.0


In [28]:
portfolio_variance([0.3, 0.3, 0.2, 0.1, 0.1], stds, corr)

1.9418933573360928

In [29]:
portfolio_variance(best_weights, stds, corr)

1.276154732833452

In [30]:
min_op = minimize(portfolio_variance, [0.2, 0.2, 0.2, 0.2, 0.2], args = (stds, corr),
         method = "SLSQP", constraints = cons, bounds = bnds)

In [31]:
best_weights = min_op.x.reshape(-1, 1)

In [32]:
min_variance = min_op.fun

In [33]:
print(f"Best Weights to achieve minimum portfolio variance: {best_weights}")
print(f"Minimum value of porfolio variance: {float(min_variance)}")

Best Weights to achieve minimum portfolio variance: [[0.001  ]
 [0.24975]
 [0.24975]
 [0.24975]
 [0.24975]]
Minimum value of porfolio variance: 0.0064729778577869825


In [34]:
print(np.sum(best_weights))

1.0000000000000246


In [35]:
portfolio_variance([0.2, 0.2, 0.2, 0.2, 0.2], stds, corr)

1.2945955715573954

In [36]:
portfolio_variance(best_weights, stds, corr)

1.393618116369442